In [1]:
# imports
import pickle, requests, pathlib, re, datetime
import fmrest

# hide ssl warnings
requests.packages.urllib3.disable_warnings()

In [2]:
# set the path to the pkl file holding the auth uname/pword
auth_pkl_path = pathlib.Path(".").resolve().parent.parent/"filemaker_db_auth.pkl"

In [3]:
# read the auth uname/pword from the pickle file
with open(auth_pkl_path,"rb") as handle:
    auth_dict = pickle.load(handle)

# create and authenticate to the server connected to the "Experiment" layout
fms = fmrest.Server(
    "https://10.173.38.223",
    user=auth_dict["uname"],
    password=auth_dict["pword"],
    database="Laser Shock",
    layout="Experiment",
    verify_ssl=False,
    api_version="v1",
)
fms.login()

'b99bfb4b7960e6a795ea9b214f7d9f6587ca50aac746b3b7c1b'

In [4]:
n_max_records = 10
foundset = fms.get_records(limit=n_max_records)
record = foundset[0]
for key, value in zip(record.keys(),record.values()) :
    keyfield = f"{repr(key)}:"
    pythonkey = key.lower().replace(" ","_")
    valfield = f"{repr(value)}:"
    valtypefield = f"({type(value)})"
    pad_to = 30
    print(f"{pythonkey: <{pad_to}} {keyfield: <{pad_to}} {valfield: <{pad_to}} {valtypefield: <{pad_to}}")

performed_by                   'Performed By':                'Lezcano':                     (<class 'str'>)               
date                           'Date':                        '10/11/2021':                  (<class 'str'>)               
energy                         'Energy':                      750:                           (<class 'int'>)               
theoretical_beam_diameter      'Theoretical Beam Diameter':   1.6666666666666667:            (<class 'float'>)             
fluence                        'Fluence':                     34.377467707849384:            (<class 'float'>)             
beam_shaper_input_beam_diameter 'Beam Shaper Input Beam Diameter': 25:                            (<class 'int'>)               
beam_shaper                    'Beam Shaper':                 'Silios':                      (<class 'str'>)               
effective_focal_length         'Effective Focal Length':      250:                           (<class 'int'>)               
dri

In [5]:
bmp_paths_txt_file_path = pathlib.Path("/Users/margareteminizer/Desktop/bmp_paths.txt")
with open(bmp_paths_txt_file_path,"r") as fp:
    bmp_rel_filepaths = [pathlib.Path(line.strip()) for line in fp.readlines()]
print(len(bmp_rel_filepaths))
print(bmp_rel_filepaths[0])

198679
20221111/TC--20221111--00027.bmp


In [6]:
tc_filename_regex = re.compile("^TC--20\d{2}(0\d|1[0-2])([0-2]\d|3[0-1])--\d{5}.bmp$")
hs_tag_regex = re.compile("^HS--20\d{2}(0\d|1[0-2])([0-2]\d|3[0-1])--\d{5}$")
datestamp_regex = re.compile("^20\d{2}_(0\d|1[0-2])_([0-2]\d|3[0-1])$")
camera_filename_regex = re.compile("^Camera_([0-1]\d|2[0-3])_[0-5]\d_[0-5]\d$")

ids_by_fp = {}

for rel_filepath in bmp_rel_filepaths:
    # "TC" filenames get parsed with everything else
    if tc_filename_regex.match(rel_filepath.name):
        date_str = rel_filepath.name.split("--")[1]
        ids_by_fp[rel_filepath] = {
            "datestamp": datetime.datetime.strptime(date_str, "%Y%m%d"),
            "counter": rel_filepath.name[:-len(".bmp")].split("--")[-1],
        }
        continue
    parts = rel_filepath.parts
    # first look for exactly one thing in the path like "HS--(8-digit datestamp)--(5-digit counter)"
    hs_parts = [part for part in parts if hs_tag_regex.match(part)]
    if len(hs_parts) == 1:
        date_str = hs_parts[0].split("--")[1]
        ids_by_fp[rel_filepath] = {
            "datestamp": datetime.datetime.strptime(date_str, "%Y%m%d"),
            "counter": hs_parts[0].split("--")[-1],
        }
        continue
    # next look for exactly one thing in the path like "yyyy_mm_dd"
    datestamp_parts = [part for part in parts if datestamp_regex.match(part)]
    camera_filename_parts = [
        part for part in parts if camera_filename_regex.match(part)
    ]
    if len(datestamp_parts) == 1 and len(camera_filename_parts) == 1:
        ids_by_fp[rel_filepath] = {
            "datestamp": datetime.datetime.strptime(datestamp_parts[0], "%Y_%m_%d"),
            "camera_filename": "_".join(camera_filename_parts[0].split("_")[1:]),
        }
        continue
    print(f"unhandled path: {rel_filepath}")
n_with_counter = len([fp for fp in ids_by_fp if "counter" in ids_by_fp[fp]])
n_with_camera_filename = len(
    [fp for fp in ids_by_fp if "camera_filename" in ids_by_fp[fp]]
)
n_total = n_with_counter + n_with_camera_filename
n_missing = len(bmp_rel_filepaths)-n_total
msg = (
    f"Found {n_with_counter} paths with a counter identified and "
    f"{n_with_camera_filename} paths with a camera filename "
    f"({n_total} total, {n_missing} missing)"
)
print(msg)

Found 127511 paths with a counter identified and 71168 paths with a camera filename (198679 total, 0 missing)


In [7]:
queries = []
for ii, (rel_filepath, id_dict) in enumerate(ids_by_fp.items()):
    #if ii not in range(11054,11055):
    #    continue
    query = {"Date":id_dict["datestamp"].strftime("%-m/%-d/%Y")}
    if "counter" in id_dict:
        query["Experiment Day Counter"] = id_dict["counter"]
    elif "camera_filename" in id_dict:
        query["Camera Filename"] = id_dict["camera_filename"]
    if query not in queries:
        queries.append(query)
foundset = fms.find(queries,limit=len(queries))
n_found = 0
for record in foundset:
    n_found+=1
print(f"found {n_found} total records from {len(queries)} total queries")


found 533 total records from 548 total queries
